<a href="https://colab.research.google.com/github/szambetti/INVEST_IO/blob/master/stock_scraping_investing_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Questo notebook scarica i vari calendari da investing, le date sono settabili come riportato sotto



CALENDARIO ECONOMICO

In [ ]:
#mounts current gdrive in colab after providing success code
#INSERISCI IL CODICE DI SUCCESSO O NON FUNZIONERA'
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/INVEST-IO/'

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
import pandas
import json

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
#importa la data di oggi nel formato corretto
from datetime import date
import time
today = date.today()
today_us_format = today.strftime("%m/%d/%Y")

In [ ]:
#function per cliccare pulsanti su chrome .click() di selenium non funziona
def click_on_button_id(x):
  element = wd.find_element_by_id(x)
  wd.execute_script("arguments[0].click();", element)

In [ ]:
#CALENDARIO ECONOMICO

url = 'https://www.investing.com/economic-calendar'

wd.set_window_size(1920,1080)
wd.maximize_window()
wd.get(url)

#apply filter on dates, scarica sempre da startdate a oggi, !!! FORMATO DATE US !!!
startdate = '08/01/2020'

#clicca sui pulsanti per filtrare la data
click_on_button_id('datePickerToggleBtn')
click_on_button_id('startDate')
wd.find_element_by_id("startDate").clear()
wd.find_element_by_id("startDate").send_keys(startdate)
click_on_button_id('endDate')
wd.find_element_by_id("endDate").clear()
wd.find_element_by_id("endDate").send_keys(today_us_format)
click_on_button_id("applyBtn")

#wait for the apply button to be effective
time.sleep(7)


#scrolla fino alla fine della pagina, aspetta questo timeout per caricare i dati
SCROLL_PAUSE_TIME = 5

#scrollare main method
last_height = wd.execute_script("return document.body.scrollHeight")
while True:
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = wd.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


list_of_news = []

#launch main scraping loop
for tr in wd.find_elements_by_xpath('//*[@id="economicCalendarData"]/tbody//tr'):
    tds = tr.find_elements_by_tag_name('td')
    list_of_news_data=[]
    for td in tds:
      if td.text != '':
        list_of_news_data.append(td.text)
      elif td.get_attribute('title') != '':
        list_of_news_data.append(td.get_attribute('title'))
      else:
        list_of_news_data.append('')
    if len(list_of_news_data) == 1:
        date = list_of_news_data[0]
    elif len(list_of_news_data) == 4:
      list_of_news.append({'date' : date, 'time': list_of_news_data[0], 
                            'currency' : list_of_news_data[1],
                            'importance' : list_of_news_data[2], 
                            'event' : list_of_news_data[3],
                            'actual' : '',
                            'expected' : '',
                            'previous' : ''})
    else:
      list_of_news.append({'date' : date, 'time': list_of_news_data[0], 
                            'currency' : list_of_news_data[1],
                            'importance' : list_of_news_data[2], 
                            'event' : list_of_news_data[3],
                            'actual' : list_of_news_data[4],
                            'expected' : list_of_news_data[5],
                            'previous' : list_of_news_data[6]})

In [ ]:
from pandas import DataFrame
df = DataFrame (list_of_news)
df.head()
df.to_csv('sr_economic_calendar.csv')

In [ ]:
df.head(15)

CALENDARIO EARNINGS

In [ ]:
#CALENDARIO EARNINGS

url = 'https://www.investing.com/earnings-calendar/'

wd.set_window_size(1920,1080)
wd.maximize_window()
wd.get(url)

#apply filter on dates, scarica sempre da startdate a oggi, !!! FORMATO DATE US !!!
startdate = '08/01/2020'

#clicca sui pulsanti per filtrare la data
click_on_button_id('datePickerToggleBtn')
click_on_button_id('startDate')
wd.find_element_by_id("startDate").clear()
wd.find_element_by_id("startDate").send_keys(startdate)
click_on_button_id('endDate')
wd.find_element_by_id("endDate").clear()
wd.find_element_by_id("endDate").send_keys(today_us_format)
click_on_button_id("applyBtn")

#wait for the apply button to be effective
time.sleep(7)


#scrolla fino alla fine della pagina, aspetta questo timeout per caricare i dati
SCROLL_PAUSE_TIME = 5

#scrollare main method
last_height = wd.execute_script("return document.body.scrollHeight")
while True:
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = wd.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


wd.save_screenshot('screen.png')

output_list = []

#main scraper loop
for tr in wd.find_elements_by_xpath('//*[@id="earningsCalendarData"]/tbody//tr'):
  tds = tr.find_elements_by_tag_name('td')
  temp_list = []
  for td in tds:
    if td.text != '':
      temp_list.append(td.text)
    elif td.get_attribute('title') != '':
      temp_list.append(td.get_attribute('title'))
    else:
      temp_list.append('')
  if len(temp_list) > 1:
    output_list.append({'date' : date, 'company': temp_list[1], 
                          'EPS' : temp_list[2],
                          'EPS Forecast' : temp_list[3], 
                          'Revenue' : temp_list[4],
                          'Revenue Forecast' : temp_list[5],
                          'Market Cap' : temp_list[6]})

In [ ]:
from pandas import DataFrame
df = DataFrame (output_list)
df.to_csv('sr_earnings_calendar.csv')

In [ ]:
df.head(15)

,date,company,EPS,EPS Forecast,Revenue,Revenue Forecast,Market Cap
0,"Thursday, August 6, 2020",Caribbean Utilities Co (CUPu),0.13,/ 0.18,45.04M,/ 53.51M,504.24M
1,"Thursday, August 6, 2020",Moneygram (MGI),0.01,/ -0.06,279.8M,/ 255.82M,218.66M
2,"Thursday, August 6, 2020",Energy Fuels Inc (UUUU),-0.08,/ -0.05,400K,/ 850K,183.42M
3,"Thursday, August 6, 2020",Heineken NV (HEINY),0.39,/ --,9.2B,/ --,51.90B
4,"Thursday, August 6, 2020",Heineken (0O26),--,/ --,--,/ --,44.03B
5,"Thursday, August 6, 2020",Hannover Re (HVRRY),0.5,/ 0.96,7.24B,/ 6.75B,19.70B
6,"Thursday, August 6, 2020",Hannover Rueckversicherung SE (0M9A),0.85,/ 1.7,6.17B,/ 5.87B,16.71B
7,"Thursday, August 6, 2020",Varian (VAR),0.78,/ 0.65,694.3M,/ 684.59M,15.81B
8,"Thursday, August 6, 2020",ASE Industrial ADR (ASX),0.11,/ 0.08,3.67B,/ 3.49B,10.19B
9,"Thursday, August 6, 2020",MTU Aero (0FC9),0.63,/ 0.54,776M,/ 751.46M,7.83B


CALENDARIO DIVIDENDS

In [ ]:
#CALENDARIO DIVIDENDS

url = 'https://www.investing.com/dividends-calendar/'

wd.set_window_size(1920,1080)
wd.maximize_window()
wd.get(url)

#apply filter on dates, scarica sempre da startdate a oggi, !!! FORMATO DATE US !!!
startdate = '08/01/2020'

#clicca sui pulsanti per filtrare la data
click_on_button_id('datePickerToggleBtn')
click_on_button_id('startDate')
wd.find_element_by_id("startDate").clear()
wd.find_element_by_id("startDate").send_keys(startdate)
click_on_button_id('endDate')
wd.find_element_by_id("endDate").clear()
wd.find_element_by_id("endDate").send_keys(today_us_format)
click_on_button_id("applyBtn")

#wait for the apply button to be effective
time.sleep(7)


#scrolla fino alla fine della pagina, aspetta questo timeout per caricare i dati
SCROLL_PAUSE_TIME = 5

#scrollare main method
last_height = wd.execute_script("return document.body.scrollHeight")
while True:
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = wd.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


wd.save_screenshot('screen.png')

output_list = []

#main scraper loop
for tr in wd.find_elements_by_xpath('//*[@id="dividendsCalendarData"]/tbody//tr'):
  tds = tr.find_elements_by_tag_name('td')
  temp_list = []
  for td in tds:
    if td.text != '':
      temp_list.append(td.text)
    elif td.get_attribute('data-value') != '':
      temp_list.append(td.get_attribute('data-value'))
    else:
      temp_list.append('')
  if len(temp_list) > 1:
  if temp_list[4] = '1':
    div_type = 'Monthly'
  elif temp_list[4] = '2':
    div_type = 'Quartely'
    elif temp_list[4] = '3':
    div_type = 'Semi-annual'
    elif temp_list[4] = '4':
    div_type = 'Annual'
    elif temp_list[4] = '5':
    div_type = 'Trailing Twelve Months'
    output_list.append({'company': temp_list[1], 
                          'Ex-dividend date' : temp_list[2],
                          'Dividend' : temp_list[3], 
                          'Type' : temp_list[4],
                          'Payment date' : temp_list[5],
                          'Yield' : temp_list[6]})

print(output_list)

[{'company': '1st Source (SRCE)', 'Ex-dividend date': 'Aug 03, 2020', 'Dividend': '0.28', 'Type': '2', 'Payment date': 'Aug 14, 2020', 'Yield': '3.32%'}, {'company': 'Bank of Montreal (BMO)', 'Ex-dividend date': 'Aug 03, 2020', 'Dividend': '0.7917', 'Type': '2', 'Payment date': 'Aug 26, 2020', 'Yield': '5.67%'}, {'company': 'Banner (BANR)', 'Ex-dividend date': 'Aug 03, 2020', 'Dividend': '0.41', 'Type': '2', 'Payment date': 'Aug 13, 2020', 'Yield': '4.58%'}, {'company': 'Blueknight (BKEP)', 'Ex-dividend date': 'Aug 03, 2020', 'Dividend': '0.04', 'Type': '2', 'Payment date': 'Aug 14, 2020', 'Yield': '11.19%'}, {'company': 'Chow Tai Fook Jewellery Group (CJEWY)', 'Ex-dividend date': 'Aug 03, 2020', 'Dividend': '0.308722', 'Type': '5', 'Payment date': 'Sep 04, 2020', 'Yield': '2.95%'}, {'company': 'ConAgra Foods (CAG)', 'Ex-dividend date': 'Aug 03, 2020', 'Dividend': '0.2125', 'Type': '2', 'Payment date': 'Sep 03, 2020', 'Yield': '2.27%'}, {'company': 'Crossamerica Partners LP (CAPL)', 'E

In [ ]:
from pandas import DataFrame
df = DataFrame (output_list)
df.to_csv('sr_dividends_calendar.csv')

In [ ]:
df.head(15)

,company,Ex-dividend date,Dividend,Type,Payment date,Yield
0,1st Source (SRCE),"Aug 03, 2020",0.28,2,"Aug 14, 2020",3.32%
1,Bank of Montreal (BMO),"Aug 03, 2020",0.7917,2,"Aug 26, 2020",5.67%
2,Banner (BANR),"Aug 03, 2020",0.41,2,"Aug 13, 2020",4.58%
3,Blueknight (BKEP),"Aug 03, 2020",0.04,2,"Aug 14, 2020",11.19%
4,Chow Tai Fook Jewellery Group (CJEWY),"Aug 03, 2020",0.308722,5,"Sep 04, 2020",2.95%
5,ConAgra Foods (CAG),"Aug 03, 2020",0.2125,2,"Sep 03, 2020",2.27%
6,Crossamerica Partners LP (CAPL),"Aug 03, 2020",0.525,2,"Aug 11, 2020",15.78%
7,East West Bancorp (EWBC),"Aug 03, 2020",0.275,2,"Aug 17, 2020",3.08%
8,Equitrans Midstream (ETRN),"Aug 03, 2020",0.15,2,"Aug 13, 2020",5.80%
9,First Trust New Opp. MLP & Energy (FPL),"Aug 03, 2020",0.0375,1,"Aug 17, 2020",11.28%
